In [1]:
!pip install transformers datasets scikit-learn --quiet

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, Trainer, TrainingArguments
import torch
import os, time
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [4]:
# Load dataset
df = pd.read_csv('/content/Reviews.csv', on_bad_lines='warn', engine='python')
df = df[['Text', 'Score']].dropna()
df = df[df['Score'] != 3]
df['label'] = df['Score'].apply(lambda x: 1 if x > 3 else 0)

In [5]:
df = df.sample(5000)

In [6]:
df['label'].value_counts()

,count
label,
1,4228
0,772


In [7]:
# Step 4: Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(df['Text'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42)


In [8]:
# Step 5: Tokenize with XLM-RoBERTa
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
# Step 6: Format as Dataset object
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)


In [13]:
# Step 7: Load the XLM-RoBERTa model
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Step 8: Define metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


In [20]:
# Step 9: Set up Trainer
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/XLM-RoBERTa_Sentiment/results',
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='/content/drive/MyDrive/XLM-RoBERTa_Sentiment/logs',
    logging_steps=10,
    load_best_model_at_end=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [21]:
start_time = time.time()
trainer.train()
training_time = time.time() - start_time
print(f"Training time: {training_time:.2f} seconds")

# Evaluation
start_test = time.time()
predictions = trainer.predict(val_dataset)
test_time = time.time() - start_test

y_pred = predictions.predictions.argmax(axis=-1)
y_true = val_labels[:len(y_pred)]

acc = accuracy_score(y_true, y_pred)
prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')

print(f"Accuracy: {acc}")
print(f"Precision: {prec}")
print(f"Recall: {rec}")
print(f"F1 Score: {f1}")
print(f"Training Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_time:.2f} seconds")

# Step 11: Evaluate the model
eval_result = trainer.evaluate()
print(eval_result)

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.305600,0.190891,0.930000,0.931405,0.933551,0.930000
2,0.196400,0.200819,0.936000,0.936337,0.936724,0.936000
3,0.090300,0.213713,0.943000,0.943941,0.945398,0.943000


Training time: 703.61 seconds


Accuracy: 0.93
Precision: 0.9675090252707581
Recall: 0.9492325855962219
F1 Score: 0.9582836710369488
Training Time: 703.61 seconds
Testing Time: 13.57 seconds


{'eval_loss': 0.19089080393314362, 'eval_accuracy': 0.93, 'eval_f1': 0.9314053998030782, 'eval_precision': 0.9335511503214918, 'eval_recall': 0.93, 'eval_runtime': 14.3803, 'eval_samples_per_second': 69.539, 'eval_steps_per_second': 4.381, 'epoch': 3.0}


In [17]:
# Save the model
model_save_path = "/content/drive/MyDrive/XLM-RoBERTa_Sentiment/saved_model"
trainer.save_model(model_save_path)

# Save the tokenizer
tokenizer.save_pretrained(model_save_path)

print(f"Model and tokenizer saved to {model_save_path}")

Model and tokenizer saved to /content/drive/MyDrive/XLM-RoBERTa_Sentiment/saved_model
